<span style="color:#1976d2; font-size:2.2em; font-weight:bold;">🔥 Projet London Fire Response</span>

<span style="color:#42a5f5; font-size:2em; font-weight:bold;">Exploration des données</span>

Ce notebook a pour but de charger, assembler et analyser rapidement les données brutes provenant de deux sources :

-les incidents signalés 

-les mobilisations des pompiers

<span style="color:#e91e63; font-size:1em; font-weight:bold;">1. Import des bibliothèques Python</span>

In [1]:
import os                # Pour manipuler les chemins de fichiers
import pandas as pd      # Pour charger et analyser les données
import glob              # Pour lister automatiquement tous les fichiers d’un dossier
import matplotlib.pyplot as plt  # Visualisation
import seaborn as sns             # Visualisation statistique

In [2]:
# Pour afficher toutes les colonnes et tout explorer !
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

  <span style="color:#e91e63; font-size:1em; font-weight:bold;">2. Définition les chemins propres</span>

In [3]:
# Revenir au dossier parent
base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Dossiers contenant les fichiers incidents et mobilisations
incidents_path = os.path.join(base_path, "data", "raw", "incidents")
mobilisations_path = os.path.join(base_path, "data", "raw", "mobilisations")

print("📁 Chemin incidents :", incidents_path)
print("📁 Chemin mobilisations :", mobilisations_path)

📁 Chemin incidents : /home/ubuntu/projects/london-fire-response/data/raw/incidents
📁 Chemin mobilisations : /home/ubuntu/projects/london-fire-response/data/raw/mobilisations


<span style="color:#e91e63; font-size:1em; font-weight:bold;">3. Charger tous les fichiers incidents (.csv)  </span>

In [4]:
incident_files = glob.glob(os.path.join(incidents_path, "*.csv"))

# On charge tous les fichiers et on les regroupe
incidents = pd.concat([pd.read_csv(f) for f in incident_files], ignore_index=True)

# Aperçu rapide
print("✅ Données incidents chargées !")
print(incidents.head())
print("📊 Nombre de lignes :", len(incidents))
print("📊 Nombre de colonnes :", len(incidents.columns))
print("📊 Colonnes :", incidents.columns.tolist())
print("📊 Types de données :", incidents.dtypes)




/tmp/ipykernel_6781/2886876441.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  incidents = pd.concat([pd.read_csv(f) for f in incident_files], ignore_index=True)


✅ Données incidents chargées !
  IncidentNumber DateOfCall  CalYear TimeOfCall  HourOfCall    IncidentGroup  \
0    235138081.0  01-Jan-09     2009   00:00:37           0  Special Service   
1         1091.0  01-Jan-09     2009   00:00:46           0  Special Service   
2         2091.0  01-Jan-09     2009   00:03:00           0             Fire   
3         3091.0  01-Jan-09     2009   00:04:27           0             Fire   
4         5091.0  01-Jan-09     2009   00:05:39           0             Fire   

  StopCodeDescription     SpecialServiceType PropertyCategory  \
0     Special Service                    RTC     Road Vehicle   
1     Special Service  Assist other agencies          Outdoor   
2      Secondary Fire                    NaN          Outdoor   
3      Secondary Fire                    NaN          Outdoor   
4      Secondary Fire                    NaN          Outdoor   

                                  PropertyType  \
0                                         Car  

> ## 📊 Observations de l'importation et exploration initiale des données `incidents`
* Les données ont bien été chargées (environ 1,8 million de lignes, 39 colonnes).
* Avertissement sur `IncidentNumber` : mélange de types (probablement des entiers et des chaînes).
* Beaucoup de colonnes sont de type texte (`object`), avec aussi des numériques (`int64`, `float64`), notamment pour les coûts, le nombre de pompes, les temps d’intervention, etc.
* Il y a plusieurs colonnes temporelles redondantes (`DateOfCall`, `TimeOfCall`, `HourOfCall`, `CalYear`) → à combiner.
* Les colonnes liées à la localisation sont riches : code postal, latitude/longitude, boroughs, wards…
* On observe des valeurs manquantes dans certaines colonnes (`UPRN`, `USRN`, `SecondPump...`).
* Quelques colonnes semblent redondantes ou doublées (`IncGeo_WardName` vs `IncGeo_WardNameNew`, `FRS` vs `ProperCase`).
* Il faudra convertir les dates en datetime, harmoniser les types, et potentiellement nettoyer les colonnes peu remplies ou inutiles.